## Catering Event Planner

In [4]:
import asyncio
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent, SocietyOfMindAgent
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
load_dotenv()

model_client = OpenAIChatCompletionClient(model='gpt-4o-mini')



## Inner Team

In [ ]:
# Inner Agent Team

planner_agent = AssistantAgent(
    name = 'PlannerAgent',
    model_client=model_client,
    description='An agent that interprests the user request and build a structured food menu',
    system_message="""
    You are a meticulous PlannerAgent specializing in event catering. 
    Your primary function is to interpret a user's high-level event brief, including the number of guests, dietary restrictions, event type (e.g., formal, casual), and food budget. 
    Break this information down into a structured, executable plan. 
    Your output should be a clear, actionable menu outline (e.g., "one appetizer, one main course, one dessert"). 
    Pass this plan to the 'RecipeAgent' to begin the creative process. 
    Do not create recipes yourself; your job is to set the stage.
    """
)

recipe_agent = AssistantAgent(
    name = 'RecipeAgent',
    model_client=model_client,
    description="An agent that create receipe based on the food menu",
    system_message="""
    You are a creative and knowledgeable RecipeAgent. 
    Your role is to generate detailed, appealing recipes based on the menu outline provided by the PlannerAgent. 
    Each recipe must include a list of ingredients, precise measurements, and step-by-step instructions. 
    You must adhere strictly to the dietary restrictions and event style. 
    After creating the recipes, pass the full menu to the CritiqueAgent for a quality check.
    """
)

critique_agent = AssistantAgent(
    name = 'CritiqueAgent',
    model_client=model_client,
    description = 'An agent that evaluate the menu and receipes',
    system_message="""
    You are the CritiqueAgent, a practical and experienced culinary expert. 
    Your job is to rigorously evaluate the menu and recipes from the RecipeAgent. 
    
    Check for potential issues:

    1. Feasibility: Is the menu too complex for the given time and resources?

    2. Ingredient Conflicts: Do multiple dishes require the same piece of equipment (e.g., the oven) at the same time?

    3. Budget Adherence: Does the menu align with the per-person food budget?

    4. Consistency: Does the menu flow well? Are there any odd flavor combinations?
    
    Your final output should be either an approval of the menu or a detailed list of suggestions for the RecipeAgent to correct. 
    
    Once the menu is approved, you will signal the UserProxyAgent to present the menu to the human for review.
    """
)

user_agent = UserProxyAgent(
    name = 'user',
    description='A proxy agent that represent the user',
    input_func=input
)

In [5]:
## Termination Condition
inner_team_termination = TextMentionTermination('APPROVE') | MaxMessageTermination(13)

culinary_team = RoundRobinGroupChat(
    participants=[planner_agent,recipe_agent,critique_agent, user_agent],
    termination_condition=inner_team_termination
)


In [6]:
from autogen_agentchat.ui import Console
task = '''
Please plan a menu for a formal dinner party for 30 people.
The guests have no dietary restrictions but prefer a menu that includes a mix of textures and flavors'''

await Console(culinary_team.run_stream(task = task))

---------- TextMessage (user) ----------

Please plan a menu for a formal dinner party for 30 people.
The guests have no dietary restrictions but prefer a menu that includes a mix of textures and flavors
---------- TextMessage (PlannerAgent) ----------
**Event Overview:**
- **Event Type:** Formal Dinner Party
- **Number of Guests:** 30
- **Dietary Restrictions:** None
- **Menu Preferences:** A mix of textures and flavors

**Menu Outline:**

1. **Appetizer Course:**
   - *Option 1:* Chilled Cucumber Avocado Soup with a touch of lime, served with a textural garnish of croutons.
   - *Option 2:* Stuffed Mushrooms with a creamy ricotta mixture topped with crispy breadcrumbs.

2. **Salad Course:**
   - *Option 1:* Mixed Greens Salad with candied walnuts, goat cheese, and a balsamic vinaigrette for a combination of crunch and creaminess. 
   - *Option 2:* Roasted Beet Salad with feta cheese and citrus segments for balance of earthiness and sweetness.

3. **Main Course:**
   - *Option 1:* Her

TaskResult(messages=[TextMessage(id='5917762b-2cfa-48b5-a833-602168767f01', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 8, 7, 25, 59, 475435, tzinfo=datetime.timezone.utc), content='\nPlease plan a menu for a formal dinner party for 30 people.\nThe guests have no dietary restrictions but prefer a menu that includes a mix of textures and flavors', type='TextMessage'), TextMessage(id='09f1764a-0981-4ece-a95c-4b9debdb681a', source='PlannerAgent', models_usage=RequestUsage(prompt_tokens=170, completion_tokens=396), metadata={}, created_at=datetime.datetime(2025, 8, 8, 7, 26, 7, 854283, tzinfo=datetime.timezone.utc), content='**Event Overview:**\n- **Event Type:** Formal Dinner Party\n- **Number of Guests:** 30\n- **Dietary Restrictions:** None\n- **Menu Preferences:** A mix of textures and flavors\n\n**Menu Outline:**\n\n1. **Appetizer Course:**\n   - *Option 1:* Chilled Cucumber Avocado Soup with a touch of lime, served with a textural garnish of c

## External Team

In [11]:
## Society of mind team

outer_culinary_team = SocietyOfMindAgent(
    name = 'CulinaryTeamAgent',
    team = culinary_team,
    model_client=model_client,
    description = 'A culinary team agent which will build the menu and receipes.',
    response_prompt="""
    You are the CulinaryTeamAgent, acting as a single, specialized expert within the outer team. 
    Your sole purpose is to produce a finalized, human-approved menu from the inner 'Culinary Team' workflow. 
    You do not generate recipes yourself. 
    Once the inner team has successfully created and gotten a menu approved by the human, you will provide this final menu and ingredients required to the LogisticsAgent as a complete input.
    You will also save the Complete Menu, Ingredients and Recipes as a .docx file.
    If the human rejects the menu, you will manage the inner team's revision process until a menu is approved.
    """
)


In [17]:
## Logistic Agent

logistic_agent = AssistantAgent(
    name = 'LogisticAgent',
    model_client=model_client,
    description='An agents which creates a complete blueprint.',
    system_message="""
    You are the LogisticsAgent, responsible for turning a finalized menu into a concrete action plan. 
    
    Your task is to receive the approved menu from the CulinaryTeamAgent and generate a detailed logistical blueprint. 
    
    This includes:

    1. Shopping List: A comprehensive list of all ingredients and quantities.

    2. Preparation Timeline: A step-by-step schedule from food prep to serving.

    3. Equipment List: A list of necessary cooking tools and equipment.
    
    Pass this complete logistics plan to the BudgetAgent.
    """
)

## budget agent
budget_agent = AssistantAgent(
    name = 'BudgetAgent',
    model_client=model_client,
    description='An agent which calculates total budget of the event.',
    system_message="""
    You are the BudgetAgent, a financial analyst for the event. 
    Your job is to calculate the total cost of the event based on the logistics plan and approved menu. 
    You will take the shopping list and equipment needs and generate a comprehensive financial report. 
    
    This report must include:

    1. Estimated cost of ingredients.

    2. Estimated cost of equipment rentals (if any).

    3. A final, total budget for the event.
    
    Provide this final budget report along with the logistics plan to the UserProxyAgent for final review and approval.
    """
)

final_approval = UserProxyAgent(
    name='FinalApproval',
    description='A proxy agent that represent the user for final approval',
    input_func=input
)

In [ ]:
## Termination condition
outer_team_termination = TextMentionTermination('APPROVE')

## Defining outer team
event_manager = RoundRobinGroupChat(
    participants=[outer_culinary_team, logistic_agent, budget_agent, final_approval],
    termination_condition=outer_team_termination
)



In [16]:
task = """Please plan a menu for a formal dinner party for 30 people. The budget for food is $50 per person. 
The guests have no dietary restrictions but prefer a menu that includes a mix of textures and flavors. Keep the item list to 5 items only
"""
await Console(event_manager.run_stream(task = task))

---------- TextMessage (user) ----------
Please plan a menu for a formal dinner party for 30 people. The budget for food is $50 per person. 
The guests have no dietary restrictions but prefer a menu that includes a mix of textures and flavors. Keep the item list to 5 items only

---------- TextMessage (PlannerAgent) ----------
**Event Overview:**
- Event Type: Formal Dinner Party
- Number of Guests: 30
- Food Budget: $50 per person
- Dietary Restrictions: None, but preference for a mix of textures and flavors

**Menu Outline:**

1. **Appetizer:**
   - Crispy Goat Cheese Salad Bites with Mixed Greens 
     - (Texture: Crispy from fried cheese, fresh and soft with greens)

2. **Soup:**
   - Creamy Roasted Tomato Basil Soup with Parmesan Croutons
     - (Texture: Creamy soup with crunchy croutons)

3. **Main Course:**
   - Herb-Crusted Salmon with Quinoa and Roasted Seasonal Vegetables
     - (Texture: Flaky salmon, nutty quinoa, and tender roasted veggies)

4. **Side Dish:**
   - Truffle

Error processing publish message for CulinaryTeamAgent_441cc252-b06d-4112-94c1-e67368aee5c8/441cc252-b06d-4112-94c1-e67368aee5c8
Traceback (most recent call last):
  File "c:\Users\ashut\OneDrive\Documents\study material\Agentic_AI_Krish\Projects\Autogen_AI_Projects\venv-analyzer-gpt\Lib\site-packages\autogen_core\_single_threaded_agent_runtime.py", line 606, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ashut\OneDrive\Documents\study material\Agentic_AI_Krish\Projects\Autogen_AI_Projects\venv-analyzer-gpt\Lib\site-packages\autogen_core\_base_agent.py", line 119, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ashut\OneDrive\Documents\study material\Agentic_AI_Krish\Projects\Autogen_AI_Projects\venv-analyzer-gpt\Lib\site-packages\autogen_agentchat\teams\_group_chat\_sequential_routed_agent.py", line 67, in on_message_impl
    return await super(

---------- TextMessage (CulinaryTeamAgent) ----------
### Finalized Menu for the Formal Dinner Party

#### 1. Appetizer: Crispy Goat Cheese Salad Bites with Mixed Greens
- Crispy goat cheese salad bites, served on a bed of mixed greens with cherry tomatoes and garnished with sliced bell peppers.

#### 2. Soup: Creamy Roasted Tomato Basil Soup with Parmesan Croutons
- Creamy roasted tomato basil soup, topped with crunchy Parmesan croutons.

#### 3. Main Course: Herb-Crusted Salmon with Quinoa and Roasted Seasonal Vegetables
- Herb-crusted salmon fillets served with quinoa and a medley of roasted seasonal vegetables.

#### 4. Side Dish: Truffle Mashed Potatoes
- Velvety smooth truffle mashed potatoes, garnished with fresh chives.

#### 5. Dessert: Deconstructed Chocolate Mousse with Fresh Berries and Crème Chantilly
- Deconstructed chocolate mousse layered with fresh berries and topped with crème chantilly.

### Ingredients Required

1. **Crispy Goat Cheese Salad Bites:**
   - Goat chees

RuntimeError: AttributeError: 'SocietyOfMindAgent' object has no attribute 'reset'
Traceback:
Traceback (most recent call last):

  File "c:\Users\ashut\OneDrive\Documents\study material\Agentic_AI_Krish\Projects\Autogen_AI_Projects\venv-analyzer-gpt\Lib\site-packages\autogen_agentchat\teams\_group_chat\_base_group_chat.py", line 502, in stop_runtime
    await self._runtime.stop_when_idle()

  File "c:\Users\ashut\OneDrive\Documents\study material\Agentic_AI_Krish\Projects\Autogen_AI_Projects\venv-analyzer-gpt\Lib\site-packages\autogen_core\_single_threaded_agent_runtime.py", line 851, in stop_when_idle
    await self._run_context.stop_when_idle()

  File "c:\Users\ashut\OneDrive\Documents\study material\Agentic_AI_Krish\Projects\Autogen_AI_Projects\venv-analyzer-gpt\Lib\site-packages\autogen_core\_single_threaded_agent_runtime.py", line 121, in stop_when_idle
    await self._run_task

  File "c:\Users\ashut\OneDrive\Documents\study material\Agentic_AI_Krish\Projects\Autogen_AI_Project